In [1]:
import numpy as np
import pickle
import cv2
from os import listdir
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    BatchNormalization, Conv2D, MaxPooling2D,
    Activation, Flatten, Dropout, Dense
)
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

2024-01-06 13:33:55.834885: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 13:33:55.977330: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 13:33:55.977371: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-06 13:33:55.978498: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-06 13:33:56.051607: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 13:33:56.053190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
EPOCHS = 10
INIT_LR = 1e-3
BS = 64
default_image_size = (64, 64)
directory_root = '/home/raj/Documents/project/plantvillage'
depth = 3

In [3]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, default_image_size)
            return image.astype(np.float16) / 255.0
        else:
            return np.array([])
    except Exception as e:
        print(f"Error: {e}")
        return None



In [4]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir:
        if directory == ".DS_Store":
            root_dir.remove(directory)
    for plant_folder in root_dir:
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        for disease_folder in plant_disease_folder_list:
            if disease_folder == ".DS_Store":
                plant_disease_folder_list.remove(disease_folder)
        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
            for single_plant_disease_image in plant_disease_image_list[:200]:
                if single_plant_disease_image.endswith(".jpg") or single_plant_disease_image.endswith(".JPG"):
                    image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{single_plant_disease_image}"
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")
except Exception as e:
    print(f"Error: {e}")

[INFO] Loading images ...
[INFO] Processing Potato___Early_blight ...
[INFO] Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
[INFO] Processing Tomato__Target_Spot ...
[INFO] Processing Potato___healthy ...
[INFO] Processing Pepper__bell___healthy ...
[INFO] Processing Pepper__bell___Bacterial_spot ...
[INFO] Processing Tomato__Tomato_YellowLeaf__Curl_Virus ...
[INFO] Processing Potato___Late_blight ...
[INFO] Processing Tomato__Tomato_mosaic_virus ...
[INFO] Processing Tomato_Septoria_leaf_spot ...
[INFO] Processing Tomato_healthy ...
[INFO] Processing Tomato_Early_blight ...
[INFO] Processing Tomato_Late_blight ...
[INFO] Processing Tomato_Bacterial_spot ...
[INFO] Processing Tomato_Leaf_Mold ...
[INFO] Image loading completed


In [5]:

label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer, open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [6]:

np_image_list = np.array(image_list)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state=42)


In [8]:

aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2,
    zoom_range=0.2, horizontal_flip=True,
    fill_mode="nearest"
)

In [9]:
model = Sequential([
    Conv2D(32, (3, 3), padding="same", input_shape=(64, 64, 3)),
    Activation("relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3)),
    Dropout(0.25),

    Conv2D(64, (3, 3), padding="same"),
    Activation("relu"),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding="same"),
    Activation("relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), padding="same"),
    Activation("relu"),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding="same"),
    Activation("relu"),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(1024),
    Activation("relu"),
    BatchNormalization(),
    Dropout(0.5),
    Dense(n_classes),
    Activation("softmax")
])

2024-01-06 13:36:08.843855: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 activation (Activation)     (None, 64, 64, 32)        0         
                                                                 
 batch_normalization (Batch  (None, 64, 64, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 21, 21, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 21, 21, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        1

In [11]:
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

print("[INFO] Training network...")

[INFO] Training network...


In [12]:
history = model.fit(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS,
    verbose=1,
    shuffle=False
)

Epoch 1/10
36/36 [==============================] - 12s 275ms/step - loss: 2.6961 - accuracy: 0.3134 - val_loss: 3.7179 - val_accuracy: 0.0558
Epoch 2/10
36/36 [==============================] - 8s 229ms/step - loss: 1.5895 - accuracy: 0.5282 - val_loss: 8.9720 - val_accuracy: 0.0558
Epoch 3/10
36/36 [==============================] - 7s 202ms/step - loss: 1.3131 - accuracy: 0.6020 - val_loss: 11.4236 - val_accuracy: 0.0558
Epoch 4/10
36/36 [==============================] - 7s 182ms/step - loss: 1.0955 - accuracy: 0.6589 - val_loss: 19.0943 - val_accuracy: 0.0558
Epoch 5/10
36/36 [==============================] - 7s 190ms/step - loss: 0.9431 - accuracy: 0.6944 - val_loss: 17.5376 - val_accuracy: 0.0558
Epoch 6/10
36/36 [==============================] - 7s 194ms/step - loss: 0.8346 - accuracy: 0.7305 - val_loss: 19.1312 - val_accuracy: 0.0558
Epoch 7/10
36/36 [==============================] - 7s 192ms/step - loss: 0.8154 - accuracy: 0.7435 - val_loss: 14.8394 - val_accuracy: 0.0558


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1] * 100}")

In [ ]:
print("[INFO] Saving model...")
pickle.dump(model, open('cnn_model.pkl', 'wb'))